<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/114_1_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一）
目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
Sheet 欄位：date, category, item, amount, payer

google sheet：https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing

In [24]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [25]:
import pandas as pd
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

In [26]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的資料
df.head()

,日期,時間,項目,金額,付款方式
0,2000-02-02,02:02,djd,3000,信用卡
1,2020-03-03,07:07,jiodw,3000,現金
2,2020-04-04,07:07,huids,300,現金
3,,,,20,line pay
4,2020-03-03,04:04,f,300,現金


In [27]:
from google.colab import auth
import gspread
from google.auth import default
from datetime import datetime

# 步驟 1: 進行 Google 驗證並連接到試算表
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

# 步驟 2: 取得特定的工作表 (Worksheet)
worksheet = gsheets.sheet1  # 或者使用 gsheets.worksheet('工作表名稱')


print("--- 輸入資料 ---")

# 步驟 3: 獲取使用者輸入並進行驗證
while True:
    date_str = input("請輸入日期 (格式：YYYY-MM-DD)：")
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        break
    except ValueError:
        print("日期格式不正確，請重新輸入。")

while True:
    time_str = input("請輸入時間 (格式：HH:MM)：")
    try:
        datetime.strptime(time_str, '%H:%M')
        break
    except ValueError:
        print("時間格式不正確，請重新輸入。")

while True:
    item_str = input("請輸入項目：")
    if item_str.strip() == "":
        print("項目不能為空，請重新輸入。")
    else:
        break

while True:
    amount_str = input("請輸入金額 (必須是數字)：")
    try:
        amount_float = float(amount_str)
        break
    except ValueError:
        print("金額格式不正確，請重新輸入數字。")

# 新增「付款方式」輸入欄位
payment_method = input("請輸入付款方式：")

# 步驟 4: 將驗證後的資料組合成一行清單
# 請確保您的 Google Sheet 欄位順序與此清單順序一致
new_row = [date_str, time_str, item_str, amount_float, payment_method]

print("\n--- 資料驗證成功 ---")
print(f"輸入的資料為：{new_row}")

# 步驟 5: 附加新資料到 Google 試算表
try:
    worksheet.append_rows([new_row])
    print("\n✅ 資料已成功寫入 Google 試算表！")
except Exception as e:
    print(f"\n寫入資料時發生錯誤：{e}")

--- 輸入資料 ---
請輸入日期 (格式：YYYY-MM-DD)：2020-04-04
請輸入時間 (格式：HH:MM)：-4:04
時間格式不正確，請重新輸入。
請輸入時間 (格式：HH:MM)：03:04
請輸入項目：hiew
請輸入金額 (必須是數字)：20
請輸入付款方式：line pay

--- 資料驗證成功 ---
輸入的資料為：['2020-04-04', '03:04', 'hiew', 20.0, 'line pay']

✅ 資料已成功寫入 Google 試算表！


In [28]:
try:
    # 讀取試算表中所有的記錄，包含標頭列
    existing_data = worksheet.get_all_records()

    # 將讀取的資料轉換為 Pandas DataFrame
    df = pd.DataFrame(existing_data)

    # 檢查 DataFrame 是否為空
    if df.empty:
        print("試算表中沒有任何記錄，無法計算總計。")
    else:
        # 確保「金額」欄位是數字類型，如果不是，嘗試轉換
        if '金額' in df.columns:
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

            # 確保「付款方式」欄位存在
            if '付款方式' in df.columns:
                # 依據「付款方式」分組，計算「金額」的總和
                payment_summary = df.groupby('付款方式')['金額'].sum()

                # ---- 修改列印結果的部分 ----
                print("\n--- 各付款方式的總計金額 ---")
                if payment_summary.empty:
                    print("沒有有效的金額可供加總。")
                else:
                    # 遍歷分組加總的結果並逐一列印
                    for payment_method, total_amount in payment_summary.items():
                        # 檢查是否有 NaN 值，如果有則跳過或顯示為 0
                        if pd.isna(total_amount):
                            print(f"{payment_method}: 0 (無效金額)")
                        else:
                            # 您可以選擇性地格式化金額，例如加上貨幣符號或小數點
                            print(f"{payment_method}: {total_amount:,.0f}") # : ,.0f 格式化為整數並加上千分位逗號
                # ----------------------------

            else:
                print("試算表中未找到「付款方式」欄位。")
        else:
            print("試算表中未找到「金額」欄位。")

except Exception as e:
    print(f"讀取或計算總計時發生錯誤：{e}")



--- 各付款方式的總計金額 ---
line pay: 49
信用卡: 3,000
現金: 3,600


In [ ]:
import gradio as gr
import gspread

# def submit_expense(date_str, time_str, item_str, amount_float, payment_method):
# def submit_expense(date_str, time_str, item_str, amount_float, payment_method):
  # 建立 Gradio 介面
with gr.Blocks(title="日常支出紀錄系統") as demo:
    gr.Markdown("## 歡迎使用日常支出紀錄系統")
    gr.Markdown("請在下方輸入您的消費紀錄，系統會自動寫入 Google 試算表並計算總額。")
    gr.Markdown("---")

    with gr.Row():
        date_input = gr.Textbox(label="日期", placeholder="YYYY-MM-DD", interactive=True)
        time_input = gr.Textbox(label="時間", placeholder="HH:MM", interactive=True)

    with gr.Row():
        item_input = gr.Textbox(label="項目", placeholder="例如：午餐", interactive=True)
        amount_input = gr.Number(label="金額", interactive=True)

    payment_input = gr.Textbox(label="付款方式", placeholder="例如：現金、信用卡、line pay", interactive=True)

    submit_button = gr.Button("送出紀錄")

    # 顯示結果的文字框
    output_message = gr.Textbox(label="狀態", lines=2, interactive=False, max_lines=2)
    output_summary = gr.Textbox(label="各付款方式總計", lines=5, interactive=False)

    # 將按鈕與函數關聯
    # submit_button.click(
    #     fn=submit_expense,
    #     inputs=[date_input, time_input, item_input, amount_input, payment_input],
    #     outputs=[output_message, output_summary]
    # )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d9b76811979c9c0045.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
